In [16]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
import pandas as pd
import numpy as np

def calculate_metrics(df, pnl_col='Trade PnL', returns_col='Returns',
                      z_entry_col='ZScore Entry', z_exit_col='ZScore Exit',
                      risk_free_rate=0.055):
    # Filter out trades with both entry & exit zscore > 3
    if z_entry_col in df.columns and z_exit_col in df.columns:
        df = df[~((df[z_entry_col] > 3) & (df[z_exit_col] > 3))]


    df[returns_col] = pd.to_numeric(df[returns_col], errors='coerce')
    df[pnl_col] = pd.to_numeric(df[pnl_col], errors='coerce')


    df = df.dropna(subset=[returns_col, pnl_col])

    total_trades = len(df)
    win_trades = len(df[df[pnl_col] > 0])
    loss_trades = len(df[df[pnl_col] < 0])
    win_percent = (win_trades / total_trades) * 100 if total_trades > 0 else 0

    total_pnl = df[pnl_col].sum()
    total_return = df[returns_col].sum()

    avg_return = df[returns_col].mean()
    std_return = df[returns_col].std(ddof=1)

    # Risk-free rate converted to per-trade basis (assuming 252 trading days)
    rf_per_trade = (1 + risk_free_rate) ** (1 / 252) - 1
    sharpe_ratio = ((avg_return - rf_per_trade) / std_return) * np.sqrt(252) if std_return != 0 else np.nan

    # Sortino Ratio
    downside_returns = df[df[returns_col] < 0][returns_col]
    downside_std = downside_returns.std(ddof=1)
    sortino_ratio = ((avg_return - rf_per_trade) / downside_std) * np.sqrt(252) if downside_std != 0 else np.nan

    # Profit Factor
    gross_profit = df[df[pnl_col] > 0][pnl_col].sum()
    gross_loss = abs(df[df[pnl_col] < 0][pnl_col].sum())
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.nan

    # Expectancy
    expectancy = total_pnl / total_trades if total_trades > 0 else np.nan

    # Avg Win / Avg Loss
    avg_win = df[df[pnl_col] > 0][pnl_col].mean()
    avg_loss = abs(df[df[pnl_col] < 0][pnl_col].mean())
    avg_win_loss_ratio = avg_win / avg_loss if avg_loss != 0 else np.nan

    # Max Drawdown
    cum_returns = (1 + df[returns_col]).cumprod()
    cum_roll_max = cum_returns.cummax()
    drawdown = (cum_returns - cum_roll_max) / cum_roll_max
    max_drawdown = drawdown.min()

    return {
        "Total Trades": total_trades,
        "Winning Trades": win_trades,
        "Losing Trades": loss_trades,
        "Win %": np.round(win_percent,2),
        "Total PnL": np.round(total_pnl,2),
        "Total Return %": np.round(((total_return)*100),2),
        "Expectancy": np.round(expectancy,2),
        "Avg Win / Avg Loss": np.round(avg_win_loss_ratio,2),
        "Sharpe Ratio": np.round(sharpe_ratio,2),
        "Sortino Ratio": np.round(sortino_ratio,2),
        "Profit Factor": np.round(profit_factor,2),
        "Max Drawdown %": np.round(max_drawdown*100,2)
    }



In [17]:
# Load CSVs
file1 = "correlated_pairs_by_sector_backtest_results.csv"
file2 = "final_results.csv"

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# For df1, calculate returns if missing
if 'Returns' not in df1.columns:
    df1['Returns'] = df1['Trade PnL'] / (
        abs(df1['Entry Open Price X'] * df1['Lot Size X']) +
        abs(df1['Entry Open Price Y'] * df1['Lot Size Y'])
    )

# Run metrics
metrics_df1 = calculate_metrics(df1, pnl_col='Trade PnL', returns_col='Returns',
                                 z_entry_col='ZScore Entry', z_exit_col='ZScore Exit')
metrics_df2 = calculate_metrics(df2, pnl_col='Trade_PnL', returns_col='Returns',
                                 z_entry_col='ZScore_Entry', z_exit_col='ZScore_Exit')



In [18]:
print("\nMetrics for correlated_pairs_by_sector_backtest_results.csv:")
for k, v in metrics_df1.items():
    print(f"{k}: {v}")


Metrics for correlated_pairs_by_sector_backtest_results.csv:
Total Trades: 182
Winning Trades: 89
Losing Trades: 93
Win %: 48.9
Total PnL: 101616.84
Total Return %: 111.77
Expectancy: 558.33
Avg Win / Avg Loss: 1.85
Sharpe Ratio: 2.57
Sortino Ratio: 4.88
Profit Factor: 1.77
Max Drawdown %: -23.2


In [19]:
print("\nMetrics for final_results.csv:")
for k, v in metrics_df2.items():
    print(f"{k}: {v}")



Metrics for final_results.csv:
Total Trades: 1656
Winning Trades: 851
Losing Trades: 805
Win %: 51.39
Total PnL: 1368972.64
Total Return %: 4464.75
Expectancy: 826.67
Avg Win / Avg Loss: 2.2
Sharpe Ratio: 1.0
Sortino Ratio: 1.53
Profit Factor: 2.33
Max Drawdown %: -247001389.12
